# 1. Project Overview

This project explores the strategic implications of fuel prices in Brazil using data from the National Agency of Petroleum, Natural Gas and Biofuels (ANP). Fuel price fluctuations directly impact transportation, logistics, and consumer costs, making it essential for policymakers and businesses to understand their trends and regional disparities.

The primary goal is to develop predictive machine learning models capable of forecasting average fuel prices based on historical trends and contextual variables such as region, fuel type, and economic indicators. This analysis supports strategic decision-making, enabling stakeholders to anticipate price changes and design better pricing policies or subsidy programs.

By applying data science techniques, including exploratory data analysis, feature engineering, and supervised learning algorithms, this project aims to:

- Identify regional and temporal patterns in fuel prices;
- Evaluate and compare machine learning models for price prediction;
- Provide data-driven insights and strategic recommendations;
- Highlight potential challenges in the Brazilian fuel market.

This project is part of the Strategic Thinking module at CCT College Dublin and integrates technical, business, and policy-driven perspectives.

# 2. Problem Definition

Fuel prices in Brazil vary significantly across regions and over time due to factors such as taxation policies, distribution logistics, and international oil price fluctuations. These disparities can have a substantial impact on transportation costs, inflation, and regional economic development.

This project aims to address the following key problem:

> **How can we use historical data and machine learning techniques to predict fuel prices in Brazil and identify regional and temporal patterns that support strategic decision-making?**

### Specific Research Questions:
- Which regions in Brazil consistently exhibit higher or lower fuel prices?
- Are there observable seasonal trends in fuel prices over time?
- Which machine learning models are most effective in predicting future prices?
- What insights can be extracted to support public policy and regulatory actions?

The answers to these questions will help identify cost inefficiencies, support regulatory strategies, and enable better forecasting of price trends.


# 3. Project Plan and Timeline

To ensure a structured approach, the project followed a defined plan with clearly segmented phases. Each phase included specific goals and deliverables, from data understanding to machine learning implementation and final reporting.

| Phase                        | Description                                                         | Timeline      |
|-----------------------------|----------------------------------------------------------------------|---------------|
| 1. Problem Definition       | Define business objective, research question, and hypotheses         | Week 1        |
| 2. Data Understanding       | Load, inspect, clean, and describe the dataset                       | Week 1-2      |
| 3. Exploratory Data Analysis (EDA) | Perform descriptive statistics and visualize key patterns         | Week 2-3      |
| 4. Feature Engineering      | Create new variables, encode categoricals, handle dates              | Week 3        |
| 5. Model Development        | Implement machine learning models (Linear, RF, MLP)                  | Week 4-5      |
| 6. Hyperparameter Tuning    | Optimize models using GridSearchCV with cross-validation             | Week 5-6      |
| 7. Results Evaluation       | Compare model metrics, generate strategic insights                   | Week 6        |
| 8. Documentation            | Finalize notebook, write report, create poster, and prepare GitHub   | Week 7        |

This timeline allowed for gradual development while ensuring room for iteration, model refinement, and integration of strategic insights.


# 4. Load and Inspect Dataset

In [3]:
# Libraries

# Data exploration
import pandas as pd
import numpy as np

# To create graphics
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns

# Machine Learning preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Train/test data splitting
from sklearn.model_selection import train_test_split

# Handle class imbalance
from sklearn.utils import class_weight

# Random operations
import random

# Deep Learning framework
import tensorflow as tf

# Format numerical outputs
pd.options.display.float_format = '{:.2f}'.format


In [4]:
# Load the dataset
df = pd.read_excel("Gas_prices_Brazil.xlsx")

# Display basic information
display("Shape of the dataset:", df.shape)
display("\nData types and non-null counts:")
display(df.info())


'Shape of the dataset:'

(58188, 17)

'\nData types and non-null counts:'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58188 entries, 0 to 58187
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   START DATE                       58188 non-null  datetime64[ns]
 1   REGION                           58188 non-null  object        
 2   STATE                            58188 non-null  object        
 3   PRODUCT                          58188 non-null  object        
 4   NUMBER OF STATIONS SURVEYED      58188 non-null  int64         
 5   UNIT OF MEASUREMENT              58188 non-null  object        
 6   AVERAGE RESALE PRICE             58188 non-null  float64       
 7   STANDARD DEVIATION RESALE        58188 non-null  float64       
 8   MINIMUM RESALE PRICE             58188 non-null  float64       
 9   MAXIMUM RESALE PRICE             58188 non-null  float64       
 10  AVERAGE RESALE MARGIN            58188 non-null  float64  

None

# 5. Dataset Description

The dataset used in this project was sourced from the Brazilian National Agency of Petroleum, Natural Gas and Biofuels (ANP). It contains detailed monthly records of fuel prices across all Brazilian states and regions.

### 📌 Key Attributes:
- `start_date`: The beginning date of the fuel price collection period.
- `region`: The geographic region in Brazil (e.g., North, South, Southeast).
- `state`: The specific state where prices were recorded.
- `fuel`: Type of fuel (e.g., Gasoline, Diesel, Ethanol, GNV).
- `unit`: Measurement unit (e.g., R$/liter, R$/kg, R$/m³).
- `avg_price`: Average resale price of the fuel.
- `sd_price`: Standard deviation of resale prices.
- `min_price`, `max_price`: Minimum and maximum prices recorded.
- `avg_price_margin`: Difference between resale and distribution price.
- `coef_price`: Coefficient of variation for resale prices.
- `dist_avg_price`: Average distribution price to fuel stations.
- `dist_sd_price`, `dist_min_price`, `dist_max_price`: Stats on distribution prices.
- `coef_dist`: Coefficient of variation for distribution prices.
- `month`, `year`, `month-year`: Extracted temporal features for trend analysis.

### 🧮 Dataset Summary:
- **Time Span:** Monthly data from multiple years (to be confirmed by `.min()` and `.max()`).
- **Observations:** Over 20,000 rows covering all regions and main fuel types.
- **Granularity:** Regional and state-level resolution.
- **Source:** [ANP Open Data](https://www.gov.br/anp/pt-br)

This dataset enables both temporal and spatial analysis of fuel price behavior, as well as predictive modeling through machine learning techniques.


In [5]:
# Preview the first 5 rows
df.head()

,START DATE,REGION,STATE,PRODUCT,NUMBER OF STATIONS SURVEYED,UNIT OF MEASUREMENT,AVERAGE RESALE PRICE,STANDARD DEVIATION RESALE,MINIMUM RESALE PRICE,MAXIMUM RESALE PRICE,AVERAGE RESALE MARGIN,RESALE COEFFICIENT OF VARIATION,AVERAGE PRICE DISTRIBUTION,STANDARD DEVIATION DISTRIBUTION,MINIMUM PRICE DISTRIBUTION,MAXIMUM PRICE DISTRIBUTION,COEF OF VARIATION DISTRIBUTION
0,2018-01-07,WEST CENTER,DISTRITO FEDERAL,HYDRATED ETHANOL,39,R$/l,3.43,0.13,3.20,3.60,0.27,0.04,3.17,0.06,3.07,3.26,0.02
1,2018-01-07,WEST CENTER,GOIAS,HYDRATED ETHANOL,234,R$/l,2.96,0.14,2.75,3.39,0.41,0.05,2.55,0.16,2.37,3.05,0.06
2,2018-01-07,WEST CENTER,MATO GROSSO,HYDRATED ETHANOL,156,R$/l,2.55,0.16,2.25,3.09,0.31,0.06,2.24,0.07,2.10,2.39,0.03
3,2018-01-07,WEST CENTER,MATO GROSSO DO SUL,HYDRATED ETHANOL,86,R$/l,3.24,0.16,3.09,3.62,0.51,0.05,2.72,0.13,2.55,3.00,0.05
4,2018-01-07,WEST CENTER,DISTRITO FEDERAL,REGULAR GASOLINE,47,R$/l,4.17,0.14,3.89,4.60,0.29,0.03,3.88,0.07,3.73,4.00,0.02


In [9]:
# Clean column names: lowercase and remove leading/trailing spaces
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

In [10]:
# Convert 'start_date' to datetime format if not already
df['start_date'] = pd.to_datetime(df['start_date'])

# Check date range
print("Date range in the dataset:")
print("Start:", df['start_date'].min())
print("End:", df['start_date'].max())


Date range in the dataset:
Start: 2018-01-07 00:00:00
End: 2024-11-03 00:00:00


In [11]:
# Statistical summary
df.describe()

,start_date,number_of_stations_surveyed,average_resale_price,standard_deviation_resale,minimum_resale_price,maximum_resale_price,average_resale_margin,resale_coefficient_of_variation,average_price_distribution,standard_deviation_distribution,minimum_price_distribution,maximum_price_distribution,coef_of_variation_distribution
count,58188,58188.00,58188.00,58188.00,58188.00,58188.00,58188.00,58188.00,58188.00,58188.00,58188.00,58188.00,58188.00
mean,2021-08-01 02:06:58.725510400,126.63,18.77,1.17,16.39,21.88,1.19,0.04,4.40,0.30,3.87,5.03,0.01
min,2018-01-07 00:00:00,1.00,1.99,0.00,1.76,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2019-11-10 00:00:00,27.00,3.89,0.14,3.61,4.40,0.00,0.03,0.00,0.00,0.00,0.00,0.00
50%,2021-09-12 00:00:00,54.00,5.10,0.21,4.79,5.80,0.00,0.04,0.00,0.00,0.00,0.00,0.00
75%,2023-04-16 00:00:00,142.00,6.46,0.34,6.06,7.29,0.38,0.06,3.08,0.08,2.88,3.28,0.02
max,2024-11-03 00:00:00,1665.00,135.44,19.35,130.00,160.00,32.58,0.59,83.14,18.39,83.00,94.50,0.59
std,NaN,212.30,32.75,2.47,28.07,38.72,4.18,0.03,13.47,1.18,11.73,15.68,0.03


## Note on Missing Values and Standard Deviation

No missing values were found in the dataset. However, the column `start_date` is of datetime type, and therefore its standard deviation is not calculated using the `describe()` function, resulting in a NaN value. This is expected behavior and does not indicate missing data.

In [12]:
# Check for missing values
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [14]:
print("Total missing values in the dataset:", df.isnull().sum().sum())

Total missing values in the dataset: 0
